In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Initialize an empty list to hold DataFrames
frames = []
trials_removed = []

# Assuming file names are in the format 'participant1.csv', 'participant2.csv', etc.
# Adjust the range as per the number of participants
for i in range(1, 16):  # Change 9 to a higher number if you have more participants
    file_name = f'../data/p{i}/p{i}_main.csv'  # Constructs the file name
    df = pd.read_csv(file_name)  # Reads the file into a DataFrame
    df['participant'] = i  # Adds the participant number
    df = df.drop('block', axis=1)  # Drops the column you don't need    

    len1 = len(df)
    df = df[(df["move_times"] > 0.4) & (df['mean_velocity'] > 20)]
    trials_removed.append(len1 - len(df))
    frames.append(df)  # Appends the DataFrame to the list

# Concatenate all data frames
combined_df = pd.concat(frames, ignore_index=True)

[0, 5, 1, 2, 0, 5, 3, 7, 0, 0, 4, 1, 3, 4, 3]


In [3]:
combined_df.sample(5)

,trial_num,move_times,elbow_start_volts,elbow_start_pix,elbow_start_cm,elbow_start_deg,elbow_end_volts,elbow_end_pix,elbow_end_cm,elbow_end_deg,...,full_feedback,terminal_feedback,rotation_direction,rotation,rotation_angle,clamp,clamp_angle,vibration,trial_type,participant
2423,47,0.676115,4.174500,-1688.136711,-52.553833,74.691602,3.514901,23.676715,0.737086,120.468868,...,0,1,0,0,0,0,0,0,0,11
785,74,0.588030,4.238946,-1687.790005,-52.543040,70.221277,3.675048,-246.540213,-7.675109,109.395937,...,0,0,0,0,0,0,0,0,0,4
3268,184,1.006007,4.176756,-1693.987490,-52.735975,74.535141,3.447878,197.791804,6.157509,125.125049,...,0,0,0,0,0,0,0,2,0,14
805,94,1.136128,4.240879,-1692.723951,-52.696640,70.087167,3.591591,-35.014025,-1.090031,115.145438,...,0,0,0,0,0,0,0,2,0,4
231,232,0.614050,4.199956,-1691.827338,-52.668727,75.369609,3.666992,-311.588666,-9.700150,112.230264,...,0,0,0,0,0,0,0,0,0,1


In [5]:
fig, ax = plt.subplots()
sns.stripplot(
    data=combined_df, x="vibration", y="bill_depth_mm", hue="sex",
    dodge=True, alpha=.2, legend=False,
)
sns.pointplot(
    data=combined_df, x="vibration", y="bill_depth_mm", hue="sex",
    dodge=.4, linestyle="none", errorbar=None,
    marker="_", markersize=20, markeredgewidth=3,
)

ValueError: Could not interpret value `Vibration` for `x`. An entry with this name does not appear in `data`.

<Figure size 1000x600 with 0 Axes>

In [ ]:
plot_colours = [[0, 0.44705882, 0.74117647],[0.92941176, 0.69411765, 0.1254902],[0.46666667, 0.6745098 , 0.18823529], [0,0,0]]

combined_df['vibration'] = pd.Categorical(combined_df['vibration'])
mean_errors = combined_df.groupby(['participant', 'vibration']).error.mean().reset_index()
control_errors = mean_errors[mean_errors['vibration'] == 0]
control_errors = control_errors.rename(columns={'error': 'control_error'})
mean_errors = pd.merge(mean_errors, control_errors[['participant', 'control_error']], on=['participant'])
mean_errors['error_diff'] = mean_errors['error'] - mean_errors['control_error']
experimental_diffs = mean_errors[mean_errors['vibration'] != 0]
experimental_diffs = experimental_diffs[experimental_diffs['vibration'] != 0]

fig, ax = plt.subplots(figsize=[8,6])
sns.stripplot(x='vibration', y='error_diff', data=experimental_diffs, ax=ax, alpha=0.5,
              hue="vibration",dodge=True, legend=False, palette=plot_colours, 
              order=[130, 90, 50], hue_order=[3, 1, 2, 0])
sns.pointplot(x='vibration', y='error_diff', data=experimental_diffs, ax=ax, hue="vibration", dodge=0.6, linestyle='none',
              errorbar=None, marker='_', markersize=20, markeredgewidth=3, palette=plot_colours, 
              order=[130, 90, 50],  hue_order=[3, 1, 2, 0])
ax.set_xlabel('Condition', fontsize=16)
ax.set_ylabel("Error (\u00B0)", fontsize=16)
ax.set_ylim([-15, 15])
ax.set_title("Vibration Induced Constant Error", fontsize=20)
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
plt.tick_params(axis='both', which='major', labelsize=12) 
sns.despine(offset=10)
handles, labels = ax.get_legend_handles_labels()
desired_handles = handles[0:3] 
desired_labels = ['Flexor Vibration', 'Dual Vibration', 'Extensor Vibration']
# desired_labels = labels[1:4]  # Keep only the first three labels
ax.legend(desired_handles, desired_labels, frameon=False, fontsize=16)

plt.tight_layout()
plt.show()
fig.savefig('../results/const_error_plot.svg', format='svg')


In [ ]:
combined_df['vibration'] = pd.Categorical(combined_df['vibration'])
mean_errors = combined_df.groupby(['participant', 'target_pos', 'vibration']).error.std().reset_index()
control_errors = mean_errors[mean_errors['vibration'] == 0]
control_errors = control_errors.rename(columns={'error': 'control_error'})
mean_errors = pd.merge(mean_errors, control_errors[['participant', 'target_pos', 'control_error']], on=['participant', 'target_pos'])
mean_errors['error_diff'] = mean_errors['error'] - mean_errors['control_error']
experimental_diffs = mean_errors[mean_errors['vibration'] != 0]
experimental_diffs = experimental_diffs[experimental_diffs['vibration'] != 0]


fig, ax = plt.subplots(figsize=[8,6])
sns.stripplot(x='target_pos', y='error_diff', data=experimental_diffs, ax=ax, alpha=0.5,
              hue="vibration",dodge=True, legend=False, palette=plot_colours,
              order=[130, 90, 50],  hue_order=[3, 1, 2, 0])
sns.pointplot(x='target_pos', y='error_diff', data=experimental_diffs, ax=ax, hue="vibration", dodge=0.6, linestyle='none',
              errorbar=None, marker='_', markersize=20, markeredgewidth=3, palette=plot_colours,
              order=[130, 90, 50],  hue_order=[3, 1, 2, 0])
ax.set_xlabel('Target Position', fontsize=16)
ax.set_ylabel("Error (\u00B0)", fontsize=16)
ax.set_title("Vibration Induced Variable Error", fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=12) 
ax.set_ylim([-7, 7])
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
sns.despine(offset=10)
handles, labels = ax.get_legend_handles_labels()
desired_handles = handles[0:3] 
desired_labels = ['Flexor Vibration', 'Dual Vibration', 'Extensor Vibration']
# desired_labels = labels[1:4]  # Keep only the first three labels
ax.legend(desired_handles, desired_labels, frameon=False, loc='upper left', fontsize=16)

plt.tight_layout()
plt.show()
fig.savefig('../results/var_error_plot.svg', format='svg')